In [1]:
from pokerrl_env.utils import readable_card_to_int
import numpy as np
from pokerrl_env.config import Config
from pokerrl_env.view import player_view, human_readable_view
from pokerrl_env.transition import step_state, init_state
from pokerrl_env.cardlib import hand_rank,encode
from pokerrl_env.datatypes import Player,ModelActions,Positions

In [4]:
config = Config(num_players=2)

state,done,winnings,action_mask = init_state(config)
print(state.shape)

state,done,winnings,action_mask = step_state(state,ModelActions.CALL,config)

(2, 91)
(2, 91) (91,)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:


# global_state = np.zeros(config.global_state_shape)
# # set stacks for two players
# board_cards = [('A', 's'), ('K', 's'), ('Q', 's'), ('J', 's'), ('T', 's')]
# board = [readable_card_to_int(card) for card in board_cards]
# hand1 = [('A', 'h'), ('K', 'h'), ('Q', 'h'), ('J', 'h')]
# hand1 = [readable_card_to_int(card) for card in hand1]
# hand2 = [('A', 'c'), ('K', 'c'), ('9', 's'), ('8', 's')]
# hand2 = [readable_card_to_int(card) for card in hand2]
# # flatten hand1 and hand2
# hand1 = [item for sublist in hand1 for item in sublist]
# hand2 = [item for sublist in hand2 for item in sublist]
# board = [item for sublist in board for item in sublist]
# hands = [hand1,hand2]
# print(board)
# print('hands', hands)
# print(global_state.shape)
# for i in range(1, 3):
#     global_state[config.global_state_mapping[f'player_{i}_stack']] = 100
#     global_state[config.global_state_mapping[f'player_{i}_active']] = 1
#     global_state[config.global_state_mapping[f'player_{i}_position']] = i
#     global_state[config.global_state_mapping[f'player_{i}_hand_range'][0]:config.global_state_mapping[f'player_{i}_hand_range'][1]] = hands[i-1]

# board = [13, 4, 12, 4, 11, 4, 10, 4, 9, 4]
# hands = [[13, 3, 12, 3, 11, 3, 10, 3], [13, 1, 12, 1, 8, 4, 7, 4]]

# print(encode([13,3]))
# en_hand = [encode(hands[1][i*2:(i*2)+2]) for i in range(0,len(hands[1])//2)]
# en_board = [encode(board[i*2:(i*2)+2]) for i in range(0,len(board)//2)]
# print(hand_rank(en_hand,en_board))
# hand_values = []
# for i in range(1,7):
#     if global_state[config.global_state_mapping[f'player_{i}_active']] == 1:
#         hand_start = config.global_state_mapping[f'player_{i}_hand_range'][0]
#         hand_end = config.global_state_mapping[f'player_{i}_hand_range'][1]
#         player_hand = [int(h) for h in global_state[hand_start:hand_end]]
#         print('player_hand', player_hand)
#         encoded_hand = [encode(player_hand[i*2:(i*2)+2]) for i in range(0,len(player_hand)//2)]
#         hand_value = hand_rank(encoded_hand, en_board)
#         hand_values.append(Player(global_state[config.global_state_mapping[f'player_{i}_position']],
#                                     global_state[config.global_state_mapping[f'player_{i}_stack']],
#                                     global_state[config.global_state_mapping[f'player_{i}_active']],
#                                     hand_value))
# # find winners

# Testing the environment

In [ ]:
from typing import Dict, List, Tuple


def get_pots(pot_players: List[Player], total_amount_invested: Dict[int, float]) -> List[Tuple[float, List[Player]]]:
    pots = []
    while len(pot_players) > 0:
        num_players = len(pot_players)
        min_invested = total_amount_invested[min(pot_players, key=lambda x: total_amount_invested[x.position]).position]
        print('min_invested',min_invested)
        pot = 0
        involved_players = []

        print('player',pot_players)
        players_to_remove = []
        for player in pot_players:
            involved_players.append(player)
            pot += min_invested
            total_amount_invested[player.position] -= min_invested
            if total_amount_invested[player.position] == 0:
                players_to_remove.append(player)
        for player in players_to_remove:
            pot_players.remove(player)
        pots.append((pot, involved_players))
        print('pots,pot_players',pots,len(pot_players))
        if num_players == len(pot_players):
            break
    return pots

In [ ]:

def example_players():
    return [
        Player(position=1, stack=100, active=1, hand=[8, 14], hand_value=5, total_invested=100),
        Player(position=2, stack=0, active=1, hand=[8, 14], hand_value=1, total_invested=50),
        Player(position=3, stack=100, active=1, hand=[8, 14], hand_value=5, total_invested=100),
    ]

def config2():
    return Config(num_players=2)

def config3():
    return Config(num_players=3)

def example_total_amount_invested():
    return {1: 100, 2: 50, 3: 100}

def total_amount_invested_2():
    return {1: 100, 2: 50, 3: 100}

def example_pots(example_players):
    return [
        (150, [example_players[0], example_players[1],example_players[2]]),
        (100, [example_players[0], example_players[2]])
    ]

def test_simple_pot(example_players,total_amount_invested_2):
    pots = get_pots([example_players[0],example_players[2]], total_amount_invested_2)
    assert pots == [
        (200, [example_players[0],example_players[2]])
    ]

example_players = example_players()
ex_pots = example_pots(example_players)


test_simple_pot(example_players,total_amount_invested_2())

min_invested 100
player [Position:1, Stack:100, Active:1, Hand:[8, 14], Position:3, Stack:100, Active:1, Hand:[8, 14]]
player 1
player 3
pots,pot_players [(200, [Position:1, Stack:100, Active:1, Hand:[8, 14], Position:3, Stack:100, Active:1, Hand:[8, 14]])] 0


In [ ]:
# config = Config(num_players=2)
# CALL = 2
# CHECK = 1
# player_index = 1
# state,done,winnings,action_mask = init_state(config)
# print(action_mask)
# state,done,winnings,action_mask = step_state(state,CALL,config)
# print(action_mask)
# while not done:
#     print('street',state[-1,config.global_state_mapping['street']])
#     print(state.shape)
#     state,done,winnings,action_mask = step_state(state,CHECK,config)
#     print(done)
#     print(action_mask)
# print(winnings)
# # player_state = human_readable_view(state,player_index,config)
# # print(player_state)

[1 0 1 1 1 1 1 1 1 1 1]
1.0
1.0
[1 0 1 1 1 1 1 1 1 1 1]
1.0
(3, 91)
1.0
1.0
1.0
True
[1 0 1 1 1 1 1 1 1 1 1]
{6.0: 0.0}
